In [1]:
import sys
!{sys.executable} -m pip install xgboost
!{sys.executable} -m pip install lightgbm
!{sys.executable} -m pip install scipy


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Convert data to machine learning
from sklearn.preprocessing import StandardScaler, OneHotEncoder #normalize numeric & convert categorical -> numerical
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split # split data into training (80%) and testing (%20)
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor, export_text
import xgboost as xgb_model
import lightgbm as lgb_model
import shap
import joblib
import scipy.stats as stats

# Load data (adjust path as needed)
df = pd.read_csv('insurance.csv')

In [3]:
# Convert categorical variables into numerical format
df['sex'] = df['sex'].map({'male': 1, 'female': 0})
df['smoker'] = df['smoker'].map({'yes': 1, 'no': 0})

# One-hot encode 'region' -> category to number
df = pd.get_dummies(df, columns=['region'], drop_first=True) #convert region into 3 new categories (the other left can be 4)

# Define features (X) and target variable (y)
X = df.drop(columns=['charges'])  # Independent variables
y = df['charges']  # Target variable

# Log-transform the target
y_log = np.log(y)

joblib.dump(X, 'X.pkl')
joblib.dump(y, 'y.pkl')

df.head()

,age,sex,bmi,children,smoker,charges,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,False,False,True
1,18,1,33.770,1,0,1725.55230,False,True,False
2,28,1,33.000,3,0,4449.46200,False,True,False
3,33,1,22.705,0,0,21984.47061,True,False,False
4,32,1,28.880,0,0,3866.85520,True,False,False


In [4]:
# Function to calculate metrics
def calculate_metrics(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return rmse, mae, r2, mape

In [5]:
# Gradient Boosting
# Models to evaluate
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=55)
gb.fit(X, y_log)

# 5-Fold CV
y_true_gb = y  # Actual charges in original scale
log_pred_gb = cross_val_predict(gb, X, y_log, cv=5)
    
y_pred_gb = np.exp(log_pred_gb)
rmse, mae, r2, mape = calculate_metrics(y_true_gb, y_pred_gb)

r2_gb = r2

# Print results
print("Results for Gradient Boosting:")
print(f"    RMSE={rmse:.2f}, MAE={mae:.2f}, R²={r2:.4f}, MAPE={mape:.2f}%")

train_log_pred_gb = gb.predict(X)
train_y_pred_gb = np.exp(train_log_pred_gb)
train_rmse, train_mae, train_r2, train_mape = calculate_metrics(y_true_gb, train_y_pred_gb)
print("Training Results for Gradient Boosting:")
print(f"    RMSE={train_rmse:.2f}, MAE={train_mae:.2f}, R²={train_r2:.4f}, MAPE={train_mape:.2f}%")

Results for Gradient Boosting:
    RMSE=4690.20, MAE=2173.81, R²=0.8499, MAPE=17.05%
Training Results for Gradient Boosting:
    RMSE=4110.91, MAE=1832.10, R²=0.8847, MAPE=14.36%


In [6]:
# XGBoost
# Models to evaluate
xgb = xgb_model.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=55)
xgb.fit(X, y_log)

# 5-Fold CV
y_true_xgb = y  # Actual charges in original scale
log_pred_xgb = cross_val_predict(xgb, X, y_log, cv=5)
    
y_pred_xgb = np.exp(log_pred_xgb)
rmse, mae, r2, mape = calculate_metrics(y_true_xgb, y_pred_xgb)

r2_xgb = r2

# Print results
print("Results for XGBoost:")
print(f"    RMSE={rmse:.2f}, MAE={mae:.2f}, R²={r2:.4f}, MAPE={mape:.2f}%")

train_log_pred_xgb = xgb.predict(X)
train_y_pred_xgb = np.exp(train_log_pred_xgb)
train_rmse, train_mae, train_r2, train_mape = calculate_metrics(y_true_xgb, train_y_pred_xgb)
print("Training Results for XGBoost:")
print(f"    RMSE={train_rmse:.2f}, MAE={train_mae:.2f}, R²={train_r2:.4f}, MAPE={train_mape:.2f}%")

Results for XGBoost:
    RMSE=4596.56, MAE=2138.65, R²=0.8558, MAPE=16.85%
Training Results for XGBoost:
    RMSE=4199.03, MAE=1884.62, R²=0.8797, MAPE=14.73%


In [7]:
# LightGBM
# Models to evaluate
lgb = lgb_model.LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=55, verbose=-1)
lgb.fit(X, y_log)

# 5-Fold CV
y_true_lgb = y  # Actual charges in original scale
log_pred_lgb = cross_val_predict(lgb, X, y_log, cv=5)
    
y_pred_lgb = np.exp(log_pred_lgb)
rmse, mae, r2, mape = calculate_metrics(y_true_lgb, y_pred_lgb)

r2_lgb = r2

# Print results
print("Results for LightGBM:")
print(f"    RMSE={rmse:.2f}, MAE={mae:.2f}, R²={r2:.4f}, MAPE={mape:.2f}%")

train_log_pred_lgb = lgb.predict(X)
train_y_pred_lgb = np.exp(train_log_pred_lgb)
train_rmse, train_mae, train_r2, train_mape = calculate_metrics(y_true_lgb, train_y_pred_lgb)
print("Training Results for LightGBM:")
print(f"    RMSE={train_rmse:.2f}, MAE={train_mae:.2f}, R²={train_r2:.4f}, MAPE={train_mape:.2f}%")

Results for LightGBM:
    RMSE=4631.93, MAE=2165.67, R²=0.8536, MAPE=16.68%
Training Results for LightGBM:
    RMSE=4254.42, MAE=1923.80, R²=0.8765, MAPE=14.99%


In [8]:
# Ensemble (Weighting average)
# Calculate weights
w_gb = r2_gb/(r2_gb + r2_xgb + r2_lgb)
w_xgb = r2_xgb/(r2_gb + r2_xgb + r2_lgb)
w_lgb = r2_lgb/(r2_gb + r2_xgb + r2_lgb)

# Ensemble
y_pred_ensemble = w_gb*y_pred_gb + w_xgb*y_pred_xgb + w_lgb*y_pred_lgb
rmse, mae, r2, mape = calculate_metrics(y, y_pred_ensemble)

# Print results
print("Results for Ensemble:")
print(f"    RMSE={rmse:.2f}, MAE={mae:.2f}, R²={r2:.4f}, MAPE={mape:.2f}%")


Results for Ensemble:
    RMSE=4604.84, MAE=2145.24, R²=0.8553, MAPE=16.77%


In [9]:
# Save models and scaler
joblib.dump(gb, 'gb_model.pkl')
joblib.dump(xgb, 'xgb_model.pkl')
joblib.dump(lgb, 'lgb_model.pkl')

# Save weights for ensemble
weights = {'w_gb': w_gb, 'w_xgb': w_xgb, 'w_lgb': w_lgb}
joblib.dump(weights, 'ensemble_weights.pkl')

['ensemble_weights.pkl']

In [517]:
# Example: new person data as dictionary
new_person = {
    'age': 63,
    'bmi': 37.7,
    'children': 0,
    'smoker': 1,
    'sex': 1,
    'region_northwest': 0,
    'region_southeast': 0,
    'region_southwest': 0
}

# Convert to DataFrame and reorder columns to match training data
new_X = pd.DataFrame([new_person])
new_X = new_X[X.columns]  # Reorder columns to match X

# Predict log-transformed output for each model
log_gb = gb.predict(new_X)
log_xgb = xgb.predict(new_X)
log_lgb = lgb.predict(new_X)

# Combine with weights (already calculated)
log_ensemble = w_gb * log_gb + w_xgb * log_xgb + w_lgb * log_lgb

# Convert back to original scale
gb_prediction = np.exp(log_gb)
xgb_prediction = np.exp(log_xgb)
lgb_prediction = np.exp(log_lgb)
ensemble_prediction = np.exp(log_ensemble)
print(new_person)
print(f"Estimated value (GB): ${gb_prediction[0]:.2f}")
print(f"Estimated value (XGB): ${xgb_prediction[0]:.2f}")
print(f"Estimated value (LGB): ${lgb_prediction[0]:.2f}")
print(f"Estimated value (Ensemble): ${ensemble_prediction[0]:.2f}")


{'age': 63, 'bmi': 37.7, 'children': 0, 'smoker': 1, 'sex': 1, 'region_northwest': 0, 'region_southeast': 0, 'region_southwest': 0}
Estimated value (GB): $52764.05
Estimated value (XGB): $51318.18
Estimated value (LGB): $52388.83
Estimated value (Ensemble): $52151.80


In [518]:
# Total claim cost prediction
log_gb_total = gb.predict(X)
log_xgb_total = xgb.predict(X)
log_lgb_total = lgb.predict(X)

log_ensemble_total = w_gb * log_gb_total + w_xgb * log_xgb_total + w_lgb * log_lgb_total

ensemble_prediction_total = np.exp(log_ensemble_total)

print(f"Total claim cost prediction: ${sum(ensemble_prediction_total):.2f}")

[19876.23905086  2175.03468889  5472.05687917 ...  2017.59555095
  2108.35740048 28077.91006358]
Total claim cost prediction: $16922304.12


In [495]:
# Assume X, y_log, gb (your base mean/median model) are loaded

def get_bootstrap_prediction_upper_bound(model, X_train, y_train, new_X_input, num_bootstraps=500, alpha=0.95):
    bootstrap_predictions = []
    n_samples = X_train.shape[0]

    for _ in range(num_bootstraps):
        # 1. Resample the training data with replacement
        # Get random indices for this bootstrap sample
        bootstrap_indices = np.random.choice(n_samples, n_samples, replace=True)
        X_boot = X_train.iloc[bootstrap_indices]
        y_boot = y_train.iloc[bootstrap_indices] # Use .iloc for Series/DataFrame

        # 2. Train a new model on each bootstrap sample
        # IMPORTANT: Create a NEW instance of the model for each bootstrap
        # Use the same hyperparameters as your best base model (e.g., gb_model)
        if model == 'gb':
            boot_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3,
                                               loss='squared_error', random_state=np.random.randint(0, 10000)) # Random seed for each model
            boot_model.fit(X_boot, y_boot)
        elif model == 'xgb':
            boot_model = xgb_model.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3,
                                               loss='squared_error', random_state=np.random.randint(0, 10000)) # Random seed for each model
            boot_model.fit(X_boot, y_boot)
        elif model == 'lgb':
            boot_model = lgb_model.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3,
                                               loss='squared_error', random_state=np.random.randint(0, 10000)) # Random seed for each model
            boot_model.fit(X_boot, y_boot)
        # 3. Predict for the new data point
        # Ensure new_X_input is in the correct DataFrame format
        pred_log = boot_model.predict(new_X_input)[0]
        bootstrap_predictions.append(np.exp(pred_log)) # Store exponentiated predictions

    # 4. Calculate the desired quantile (upper bound)
    upper_bound_value = np.percentile(bootstrap_predictions, alpha * 100)
    return upper_bound_value, bootstrap_predictions # Return predictions list for plotting distribution

# Example usage in your Streamlit app after 'Predict Individual' button is pressed:
# Ensure new_X is prepared as a DataFrame (1 row, correct columns)
# new_X (from Streamlit inputs)
# Assuming gb_model is loaded as 'gb'
# Assuming X, y_log are loaded as the full training data

# Example:
bootstrap_ub, all_bootstrap_preds = get_bootstrap_prediction_upper_bound('gb', X, y_log, new_X, num_bootstraps=500, alpha=0.95)
print(f"Upper bound cost (Bootstrapping): ${bootstrap_ub:.2f}")


Upper bound cost (Bootstrapping): $55644.85
